# Chapter 1: Introduction to PyChiquito
Welcome to PyChiquito, a step-based high-level rust DSL (pychiquito is a python DSL for chiquito) that provides better syntax and abstraction for constraint building and column placement when writing PLONKish circuits, which supports custom gates and lookup arguments. PyChiquito has a Halo2 backend, which is a low level zkDSL that writes circuits using the PLONKish arithemtization. PyChiquito is working on supporting additional backends.

PyChiquito is composed of two parts. [The Python part](https://github.com/qwang98/PyChiquito) exposes functions that user writes circuits with. [The Rust part](https://github.com/privacy-scaling-explorations/chiquito) is called by the Python functions behind the scenes and converts what the user writes into a Halo2 circuit.

The key advantages of PyChiquito include: 
- Abstraction and simplification on the readability and learnability of Halo2 circuits.
- Composabiity with other Halo2 circuits.
- Modularity of using multiple frontends (Python and Rust) and backends (Halo2 and beyond).
- Smooth user experience with a dynamically typed language (Python).

For more on why you need PyChiquito, refer to the [Chiquito README](https://github.com/privacy-scaling-explorations/chiquito#readme) and the [Appendix](https://github.com/privacy-scaling-explorations/chiquito/blob/main/Appendix.md/#design-principles) on its design principles. For more on PLONKish concepts and Halo2 circuits, refer to the [Halo2 book](https://zcash.github.io/halo2/index.html).

# Chapter 2: Quick Start
PyChiquito requires using a Python virtual environment for its dependencies, which you should have setup following the [PyChiquito README](https://github.com/qwang98/PyChiquito#readme).

Specifically, after cloning PyChiquito, you need to run the following commands in your local repo (NOT in this Jupyter Notebook), till you see `(.env)` at the start of your command line, which means that you've successfully enabled the virtual environment. You also need to install `maturin` and `py_ecc` dependencies to your local environment and build the project using `maturin develop`. Again, these are all done in your local command line.

In [ ]:
# Create a virtual environment
python3 -m venv .env

# Activate the virtual environment
source .env/bin/activate

# Install the required packages
pip install maturin
pip install py_ecc

# Build the project
maturin develop

Now, run the following in your local command line to navigate to the `PyChiquito/pychiquito` directory which contains all the Python files and activate Jupyter Lab. Finally, in the browser, navigate to `tutorial.ipynb` to start using this interactive tutorial.

In [ ]:
# Navigate to the pychiquito Python directory
cd pychiquito

# If you haven't installed Jupyter Lab
pip install jupyterlab

# Activate Jupyter Lab
jupyter lab

Once in Jupyter Lab browser, make sure you are using the Python virtual environment rather than default for kernel.

# Chapter 3: Fibonacci Example–Part 1 (Do NOT run Chapter 3 code chunks)
The best learning is by doing. We will now walk through the [fibonacci.py](https://github.com/qwang98/PyChiquito/blob/main/pychiquito/fibonacci.py) example with the following PLONKish table layout and three signals–a, b, and c.
| a | b | c |
| - | - | - |
| 1 | 1 | 2 |
| 1 | 2 | 3 |
| 2 | 3 | 5 |
| 3 | 5 | 8 |
| ... | |   |
## Imports
These imports are for the typing hints included in this example. Python is a dynamically typed interpreted language and typings aren't enforced.

In [ ]:
from __future__ import annotations
from typing import Tuple

The following imports are required, including:
- `Circuit` and `StepType`, the most important data types, from the domain specific language (dsl).
- Equal constraint `eq` from the constraint builder (cb).
- `Queriable` type used to query signals.
- Field element `F` from utils.

In [ ]:
from dsl import Circuit, StepType
from cb import eq
from query import Queriable
from util import F

## Circuit
On the highest level, we are building a `Circuit` object in PyChiquito, that's why we start with creating a `Fibonacci` class that inherits the `Circuit` parent class. Within the `Fibonacci` class, we define two functions:
- `setup`, which defines the circuit configuration using signals and step types (more on this later).
- `trace`, which defines the circuit layout and the trace of assigning witness values.

In [ ]:
class Fibonacci(Circuit):
    def setup(self: Fibonacci):
        # TODO

    def trace(self: Fibonacci, args: Any):
        # TODO

## Circuit Setup
We `setup` circuit configuration using signals and step types. "Signals" are variables we use to express custom constraints and lookup arguments. PyChiquito is a step-based language, for which we configure different "step types". Each step type defines different relationships among the signals. Each circuit is an arbitrary combination of arbitrary instances of step types. Think of the step types as different burger ingredients, such as bun, lettuce, and patties. Think of their instantiation as combining different amounts of ingredients to make the burger, such as 1 bun, 1 lettuce, 2 patties, and 1 bun. Think of the PyChiquito circuit as the burger you create. It's that simple!

Now back to signals. Because we can assign different values to the same signal at different step instances, we can query the signal at different positions, and that's why signals are also called `Queriable` in PyChiquito. There are signals that live on the circuit top-level, called "forward signals", which we can query at different step instances. There are signals that live in a specific step type, called "internal signals".

In the following example, we add two signals, "a" and "b", to the Fibonacci circuit. Here, signal definitions start with `self.a` and `self.b`, because we append them directly to the Circuit. Because they are circuit top-level signals rather than specific to a step-type, they are forward signals created using `self.forward`.

In [ ]:
class Fibonacci(Circuit):
    def setup(self: Fibonacci):
        self.a: Queriable = self.forward("a")
        self.b: Queriable = self.forward("b")
        # TODO

Now back to step types. In the Fibonacci cicuit, we consider each row a step instance. For example, in row 1, a=1 b=1 c=2. In row 2, a=1 b=2 c=3. Given the nature of Fibonacci, We want to enforce the constraint that b in the current step instance equals a in the next step instance and that c in the current step instance equals b in the next step instance. We created "a" and "b" as forward signals, because they are referred to across multiple step instances.

We define two step types in the Fibonacci circuit, `fibo_step` and `fibo_last_step`, because the last step instance cannot enforce constraints that involve a signal in the next step instance, as there's no next step instance. More on the step type setup later. For now, we simply initiate and append them to the circuit using `self.step_type`.

In [ ]:
        # ...
        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.fibo_last_step = self.step_type(FiboLastStep(self, "fibo_last_step"))
        # TODO

Additionally, we need to constrain that the first step is indeed an instance of `fibo_step` and the last step an instance of `fibo_last_step`. We also want to enforce the total number of steps. See code below:

In [ ]:
        # ...
        self.pragma_first_step(self.fibo_step)
        self.pragma_last_step(self.fibo_last_step)
        self.pragma_num_steps(11)
        # TODO

In [ ]:
Putting the circuit `setup` together:

In [ ]:
class Fibonacci(Circuit):
    def setup(self: Fibonacci):
        self.a: Queriable = self.forward("a")
        self.b: Queriable = self.forward("b")

        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.fibo_last_step = self.step_type(FiboLastStep(self, "fibo_last_step"))

        self.pragma_first_step(self.fibo_step)
        self.pragma_last_step(self.fibo_last_step)
        self.pragma_num_steps(11)

    def trace(self: Fibonacci, args: Any):
        # TODO

## Circuit Trace
Now we finished defining the ingredients (step types). Let's make the burger (combining step instances)!

In `trace`, we define the circuit layout using step instances and a trace of witness value assignments. `trace` takes two arguments, the `Fibonacci` circuit itself and the witness value assignment arguments `args`.

We call `self.add` to instantiate a specific step type we defined (`fibo_step` or `fibo_last_step`) and pass in the witness values for "a" and "b". Note that we only hardcoded witness values for the first step instance as `(1, 1)`, because all other witness values can be calculated given the nature of Fibonacci. We didn't pass in witness values for "c", because they are only ever calculated.

Note that we need to pass in witness value assignments in a single argument `args` and therefore we use a tuple in this case. `args` can really be any data type as long as it's one single argument.

After creating the first `fibo_step` instance, we loop over `fibo_step` creation for 9 more times, each time calculating and passing in a different tuple of assignments. We close off the circuit with a `fibo_last_step` instance. Voila, here's our PyChiquito Fibonacci "burger" with 10 patties (`fibo_step`) and 1 bun (`fibo_last_step`)!

In [ ]:
class Fibonacci(Circuit):
    def setup(self: Fibonacci):
        # ...
        
    def trace(self: Fibonacci, args: Any):
        self.add(self.fibo_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 10):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a
        self.add(self.fibo_last_step, (a, b))

# Chapter 4: Fibonacci Example–Part 2 (Do NOT run Chapter 4 code chunks)
Now that we have the Fibonacci circuit configured with signals and step types using `setup` and layout defined with step instances and witness assignments using `trace`, we move forward to configuring `fibo_step` and `fibo_last_step`.
## StepType
PyChiquito provides a `StepType` parent class that we can customarily inherit. Again, for each `StepType`, we define two functions:
- `setup`, which defines the step type configuration using signals
- `wg`, which defines witness assignment within the step type

In [ ]:
class FiboStep(StepType):
    def setup(self: FiboStep):
        # TODO

    def wg(self: FiboStep, args: Tuple[int, int]):
        # TODO


class FiboLastStep(StepType):
    def setup(self: FiboLastStep):
        # TODO

    def wg(self: FiboLastStep, values=Tuple[int, int]):
        # TODO

## FiboStep Setup
As each step instance contains three signals, only two of which we've defined so far, we define a third signal "c", whose relationship is only defined within the step type and therefore an internal signal created using `self.internal`.

Next, we define constraints among signals, both forward and internal. There are two types of constraints in PyChiquito:
- `constr` stands for constraints among signals that are specific to the step type, i.e. internal signals.
- `transition` stands for constraints involving circuit-level signals, i.e. forward signals and etc.

In the code snippet below, forward signals "a" and "b" are expressed as `self.circuit.a` and `self.circuit.b`, whereas internal signal "c" is expressed `self.c`, because "a" and "b" are at the circuit-level. `self.circuit.a.next()` queries the value of circuit-level signal "a" at the next step instance. `eq` is a constraint builder that enforces equality between the two arguments passed in. The following constraints are translated as:
- a + b == c
- b == value of a in next step instance
- c = value of b in next step instance

In [ ]:
class FiboStep(StepType):
    def setup(self: FiboStep):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

## FiboStep Witness Generation
Similar to `trace`, `wg` (witness generation) defines witness value assignments at the step type level. Here, the `args` we pass in is a tuple of values for signals "a" and "b". We assign them to forward signals "a" and "b" and then their sum to internal signal "c". Note that the format of `args` in `wg` needs to match `args` in the `add` function called in `trace`, e.g. `(a, b)` in `self.add(self.fibo_step, (a, b))`. This is because `add` creates step instance by calling `wg` associated with the step type. Here in `wg`, we don't pass in any hardcoded values, because we need to make the function generic for `add` to use across different step instances.

Finally, note that in `self.assign`, `a_value` and `b_value` are both wrapped in `F`, which converts them from int to field elements. All witness assignments in PyChiquito are field elements.

In [ ]:
class FiboStep(StepType):
    def setup(self: FiboStep):
        # ...

    def wg(self: FiboStep, args: Tuple[int, int]):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

Putting everything for `FiboStep` together, we have:

In [ ]:
class FiboStep(StepType):
    def setup(self: FiboStep):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self: FiboStep, args: Tuple[int, int]):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

## FiboLastStep Setup and Witness Generation
`FiboLastStep` is identical except that we don't create transition constraints like "b == a.next", because there's no next step instance after the last.

In [ ]:
class FiboLastStep(StepType):
    def setup(self: FiboLastStep):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))

    def wg(self: FiboLastStep, values=Tuple[int, int]):
        a_value, b_value = values
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

# Chapter 5: Fibonacci Example–Part 3 (Code chunks CAN be run)
Putting everything together, let's run the `fibonacci.py` example:

In [ ]:
from __future__ import annotations
from typing import Tuple

from dsl import Circuit, StepType
from cb import eq
from query import Queriable
from util import F


class Fibonacci(Circuit):
    def setup(self: Fibonacci):
        self.a: Queriable = self.forward("a")
        self.b: Queriable = self.forward("b")

        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.fibo_last_step = self.step_type(FiboLastStep(self, "fibo_last_step"))

        self.pragma_first_step(self.fibo_step)
        self.pragma_last_step(self.fibo_last_step)
        self.pragma_num_steps(11)

    def trace(self: Fibonacci, args: Any):
        self.add(self.fibo_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 10):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a
        self.add(self.fibo_last_step, (a, b))


class FiboStep(StepType):
    def setup(self: FiboStep):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self: FiboStep, args: Tuple[int, int]):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))


class FiboLastStep(StepType):
    def setup(self: FiboLastStep):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))

    def wg(self: FiboLastStep, values=Tuple[int, int]):
        a_value, b_value = values
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

Everything we went through in the code chunk above defines how the circuit and its step types are configured and witness values assigned to them. To initiate the circuit, we call the class constructor:

In [ ]:
fibo = Fibonacci()

You can also print the circuit. In the print out, you will see the two step types and two forward signals "a" and "b" at the circuit top-level. Within each step type, you will see the internal signals and constraints. The big random looking numbers are UUIDs that we use to uniquely identify objects in the circuit, which you don't need to worry about.

In [ ]:
print(fibo)

After initiating the Fibonacci circuit, we can generate witness assignments for it. `gen_witness` takes one argument of external input with `Any` type. However, because the only external input, `(1, 1)`, was hardcoded in `trace`, we don't need to provide an additional one and can put `None` for this argument. In practice, one circuit can have many different sets of witness assignments, each generated by a different external input argument. This is why we expose the `gen_witness` function to you.

In [ ]:
fibo_witness = fibo.gen_witness(None)

Again, you can print the witness assignments:

In [ ]:
print(fibo_witness)

We can convert and register the PyChiquito circuit as a Halo2 circuit in Rust Chiquito. `ast_to_halo2` function achieves this purpose and prints out the Halo2 circuit UUID generated for our PyChiquito circuit. You don't need to do anything with the UUID.

In [ ]:
fibo.ast_to_halo2()

Finally, we can verify the proof with the witness. The print out includes Halo2 and Rust Chiquito debug messages. `Ok(())` means that proof was correctly generated and verified for the witness assignments and circuit. `Err()` prints out Halo2 and Rust Chiquito error messages, usually because some constraints in the circuit were not satisfied.

In [ ]:
fibo.verify_proof(fibo_witness)

Congratulations! Now you finished writing your first Fibonacci circuit and learned about the most essential concepts behind the step-based design of PyChiquito, which is really as easy as bun-lettuce-patty-patty-bun, basically combining step instances to a PyChiquito burger! With abstraction, composability, modularity, and smooth user experience as the key tenets, writing Halo2 circuits has never been easier with PyChiquito!